# 🎯 TP1 : Introduction à CLIMADA - Exemple Simple

## Formation DGE Côte d'Ivoire - Jour 1

### Objectifs pédagogiques :
- Comprendre les concepts de base : Aléa, Exposition, Vulnérabilité
- Créer ses premiers objets CLIMADA
- Effectuer un calcul d'impact simple
- Visualiser les résultats sur carte

### Durée : 2h00
### Niveau : Débutant

---

## 📚 Étape 1 : Imports et Configuration

In [ ]:
# Imports essentiels CLIMADA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Imports CLIMADA
from climada.hazard import Hazard
from climada.entity import Exposures, ImpactFunc, ImpactFuncSet
from climada.engine import Impact
from climada.util.coordinates import coord_on_land

# Configuration matplotlib pour affichage
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Imports réussis - CLIMADA opérationnel!")
print(f"📍 Répertoire de travail : {Path.cwd()}")

## 🌧️ Étape 2 : Création d'un Aléa Simple (Précipitations)

Nous allons créer un aléa de précipitations extrêmes pour la région d'Abidjan basé sur des données fictives mais réalistes.

In [ ]:
# Définir la zone d'étude : Abidjan et environs
# Coordonnées réelles d'Abidjan
lon_min, lat_min = -4.2, 5.2  # Sud-Ouest
lon_max, lat_max = -3.8, 5.6  # Nord-Est

# Créer grille spatiale 10x10 (résolution ~4km)
n_grid = 10
lon_grid = np.linspace(lon_min, lon_max, n_grid)
lat_grid = np.linspace(lat_min, lat_max, n_grid)
lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)

# Aplatir les coordonnées
centroids_lat = lat_mesh.flatten()
centroids_lon = lon_mesh.flatten()

print(f"📐 Grille créée : {n_grid}x{n_grid} = {len(centroids_lat)} points")
print(f"🗺️ Zone : {lon_min:.2f}° à {lon_max:.2f}°E, {lat_min:.2f}° à {lat_max:.2f}°N")
print(f"📏 Résolution approximative : {111*(lon_max-lon_min)/n_grid:.1f} km")

In [ ]:
# Créer des données d'intensité réalistes pour les précipitations
# Basé sur les observations SODEXAM (données fictives mais réalistes)

# Seed pour reproductibilité
np.random.seed(42)

# Simuler 3 événements de précipitations extrêmes
n_events = 3
event_names = ['Pluies_Juin_2018', 'Pluies_Septembre_2019', 'Pluies_Octobre_2020']
event_dates = ['2018-06-15', '2019-09-22', '2020-10-08']

# Intensités moyennes observées à Abidjan (mm/24h)
base_intensities = [180, 150, 210]  # Valeurs réalistes pour Abidjan

# Créer matrice intensités (events x centroides)
intensities = np.zeros((n_events, len(centroids_lat)))

for i, base_intensity in enumerate(base_intensities):
    # Variation spatiale réaliste (plus fort près de la côte)
    distance_to_coast = np.sqrt((centroids_lon - (-4.0))**2 + (centroids_lat - 5.3)**2)
    coastal_factor = np.exp(-distance_to_coast * 2)  # Décroissance exponentielle
    
    # Ajouter variabilité aléatoire
    random_factor = np.random.normal(1.0, 0.2, len(centroids_lat))
    
    # Intensité finale
    intensities[i, :] = base_intensity * coastal_factor * random_factor
    
    # S'assurer valeurs non négatives
    intensities[i, :] = np.maximum(intensities[i, :], 0)

print(f"💧 Intensités créées pour {n_events} événements")
print(f"📊 Intensité moyenne : {intensities.mean():.1f} mm/24h")
print(f"📊 Intensité max : {intensities.max():.1f} mm/24h")
print(f"📊 Intensité min : {intensities.min():.1f} mm/24h")

In [ ]:
# Créer l'objet Hazard CLIMADA
hazard = Hazard('FL')  # FL = Flood (inondations dues aux précipitations)

# Remplir les attributs obligatoires
hazard.centroids.set_lat_lon(centroids_lat, centroids_lon)
hazard.event_id = np.arange(1, n_events + 1)
hazard.event_name = event_names
hazard.date = pd.to_datetime(event_dates).values
hazard.frequency = np.ones(n_events) / n_events  # Fréquence annuelle simplifiée
hazard.orig = np.ones(n_events, dtype=bool)  # Tous événements originaux
hazard.intensity = intensities

# Unités et métadonnées
hazard.units = 'mm/day'
hazard.tag.description = 'Précipitations extrêmes Abidjan - Formation DGE'
hazard.tag.file_name = 'precipitations_abidjan_formation.hdf5'

# Vérifier la validité
hazard.check()

print("✅ Objet Hazard créé et validé!")
print(f"📋 Résumé Hazard:")
print(f"   - Type: {hazard.tag.haz_type}")
print(f"   - Événements: {hazard.size[0]}")
print(f"   - Centroides: {hazard.size[1]}")
print(f"   - Période: {hazard.date[0]} à {hazard.date[-1]}")

## 💰 Étape 3 : Création d'une Exposition Simple

Nous allons créer une exposition économique simplifiée pour différents secteurs à Abidjan.

In [ ]:
# Données démographiques et économiques fictives mais réalistes
# Basées sur les statistiques INS Côte d'Ivoire

# Population d'Abidjan par commune (données simplifiées)
communes_abidjan = {
    'Cocody': {'pop': 687000, 'pib_per_capita': 2800000},  # FCFA/an
    'Yopougon': {'pop': 1200000, 'pib_per_capita': 1200000},
    'Abobo': {'pop': 1300000, 'pib_per_capita': 1000000},
    'Adjamé': {'pop': 450000, 'pib_per_capita': 2200000},
    'Plateau': {'pop': 15000, 'pib_per_capita': 8500000},  # District affaires
    'Marcory': {'pop': 500000, 'pib_per_capita': 1800000},
    'Treichville': {'pop': 280000, 'pib_per_capita': 2000000},
    'Koumassi': {'pop': 430000, 'pib_per_capita': 1400000},
    'Port-Bouët': {'pop': 320000, 'pib_per_capita': 1600000},
    'Attécoubé': {'pop': 380000, 'pib_per_capita': 1100000}
}

print("📊 Données communales Abidjan (fictives mais réalistes):")
for commune, data in communes_abidjan.items():
    print(f"   {commune}: {data['pop']:,} hab, {data['pib_per_capita']:,} FCFA/hab")

total_pop = sum(data['pop'] for data in communes_abidjan.values())
print(f"\n👥 Population totale: {total_pop:,} habitants")

In [ ]:
# Créer points d'exposition distribués spatialement
# Simuler distribution spatiale réaliste de la richesse

np.random.seed(123)  # Pour reproductibilité

# Nombre de points d'exposition par centroide (densité variable)
n_exposure_points = 200

# Coordonnées exposition (légèrement décalées des centroides hazard)
exposure_lons = []
exposure_lats = []
exposure_values = []
exposure_categories = []

categories = ['Résidentiel', 'Commercial', 'Industriel', 'Infrastructure']
category_weights = [0.6, 0.2, 0.15, 0.05]  # Distribution réaliste

for i in range(n_exposure_points):
    # Position aléatoire dans la zone
    lon = np.random.uniform(lon_min, lon_max)
    lat = np.random.uniform(lat_min, lat_max)
    
    # Valeur basée sur distance au centre (Plateau plus cher)
    dist_to_plateau = np.sqrt((lon - (-4.024))**2 + (lat - 5.325)**2)
    urban_factor = np.exp(-dist_to_plateau * 8)  # Plus cher près du centre
    
    # Catégorie aléatoire
    category = np.random.choice(categories, p=category_weights)
    
    # Valeur économique selon catégorie (millions FCFA)
    if category == 'Résidentiel':
        base_value = np.random.lognormal(15, 0.8)  # 15-50M FCFA
    elif category == 'Commercial':
        base_value = np.random.lognormal(16, 1.0)  # 20-200M FCFA
    elif category == 'Industriel':
        base_value = np.random.lognormal(17, 1.2)  # 50-500M FCFA
    else:  # Infrastructure
        base_value = np.random.lognormal(18, 0.6)  # 100-300M FCFA
    
    final_value = base_value * (0.5 + urban_factor * 1.5)
    
    exposure_lons.append(lon)
    exposure_lats.append(lat)
    exposure_values.append(final_value)
    exposure_categories.append(category)

print(f"🏢 {n_exposure_points} points d'exposition créés")
print(f"💰 Valeur totale: {sum(exposure_values)/1000:.1f} milliards FCFA")
print(f"📊 Répartition par catégorie:")
for cat in categories:
    count = exposure_categories.count(cat)
    value = sum(v for v, c in zip(exposure_values, exposure_categories) if c == cat)
    print(f"   {cat}: {count} points, {value/1000:.1f} Mds FCFA")

In [ ]:
# Créer objet Exposures CLIMADA
exposures = Exposures()

# Remplir les données
exposures.gdf = gpd.GeoDataFrame({
    'value': exposure_values,
    'latitude': exposure_lats,
    'longitude': exposure_lons,
    'category': exposure_categories,
    'region_id': np.ones(len(exposure_lons), dtype=int),  # Tous dans région Abidjan
    'impf_FL': np.ones(len(exposure_lons), dtype=int)  # Impact function ID
})

# Créer géométrie Points
from shapely.geometry import Point
exposures.gdf['geometry'] = [Point(lon, lat) for lon, lat in zip(exposure_lons, exposure_lats)]

# Métadonnées
exposures.tag.description = 'Exposition économique Abidjan - Formation DGE'
exposures.tag.file_name = 'exposition_abidjan_formation.hdf5'
exposures.value_unit = 'FCFA'
exposures.ref_year = 2023
exposures.crs = 'EPSG:4326'

# Assigner centroides (mapping spatial)
exposures.assign_centroids(hazard)

# Vérifier
exposures.check()

print("✅ Objet Exposures créé et validé!")
print(f"📋 Résumé Exposures:")
print(f"   - Points: {len(exposures.gdf)}")
print(f"   - Valeur totale: {exposures.gdf.value.sum()/1e6:.0f} millions FCFA")
print(f"   - Valeur moyenne: {exposures.gdf.value.mean()/1e6:.1f} millions FCFA")
print(f"   - Année référence: {exposures.ref_year}")

## 📈 Étape 4 : Fonction de Dommage

Nous créons une fonction de dommage qui lie l'intensité des précipitations aux dommages économiques.

In [ ]:
# Créer fonction de dommage pour inondations pluviales
# Basée sur littérature scientifique et expertise locale

# Points de calibration (mm/24h -> % dommage)
# Valeurs réalistes pour contexte urbain tropical
intensities_cal = np.array([0, 50, 100, 150, 200, 250, 300, 400])  # mm/24h
damages_cal = np.array([0, 2, 8, 20, 35, 50, 65, 80])  # % de la valeur

# Fonction impact
impact_func = ImpactFunc()
impact_func.id = 1
impact_func.name = 'Inondations_pluviales_Abidjan'
impact_func.haz_type = 'FL'
impact_func.intensity_unit = 'mm/day'

# Courbe intensité-dommage
impact_func.intensity = intensities_cal
impact_func.mdd = damages_cal / 100  # Convertir % en fraction
impact_func.paa = np.minimum(damages_cal / 10, 1.0)  # Probabilité d'être affecté

# Vérifier fonction
impact_func.check()

# Créer set de fonctions d'impact
impact_func_set = ImpactFuncSet()
impact_func_set.append(impact_func)
impact_func_set.check()

print("✅ Fonction de dommage créée!")
print(f"📊 Points de calibration: {len(intensities_cal)}")
print(f"💧 Intensité max: {intensities_cal.max()} mm/24h")
print(f"💸 Dommage max: {damages_cal.max()}%")

# Visualiser la fonction
plt.figure(figsize=(10, 6))
plt.plot(intensities_cal, damages_cal, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Intensité précipitations (mm/24h)')
plt.ylabel('Dommages (% valeur exposée)')
plt.title('Fonction de Dommage - Inondations Pluviales Abidjan')
plt.grid(True, alpha=0.3)
plt.xlim(0, 400)
plt.ylim(0, 85)

# Ajouter annotations
for i, (x, y) in enumerate(zip(intensities_cal[::2], damages_cal[::2])):
    plt.annotate(f'{y}%', (x, y), xytext=(5, 5), textcoords='offset points')

plt.tight_layout()
plt.show()

## 🎯 Étape 5 : Calcul des Impacts

Maintenant nous combinons aléa, exposition et vulnérabilité pour calculer les impacts économiques.

In [ ]:
# Calculer les impacts
impact = Impact()
impact.calc(exposures, impact_func_set, hazard, save_mat=True)

print("✅ Calcul d'impact terminé!")
print(f"\n📊 RÉSULTATS GLOBAUX:")
print("-" * 40)

# Impact attendu annuel (AAI)
aai_total = impact.aai_agg  # FCFA
print(f"💰 Impact Attendu Annuel (AAI): {aai_total/1e9:.2f} milliards FCFA")

# Impacts par événement
print(f"\n🌧️ IMPACTS PAR ÉVÉNEMENT:")
print("-" * 40)

for i, (event_name, event_impact) in enumerate(zip(hazard.event_name, impact.at_event)):
    event_date = pd.to_datetime(hazard.date[i]).strftime('%d/%m/%Y')
    print(f"{event_name} ({event_date}): {event_impact/1e9:.2f} Mds FCFA")

# Statistiques
max_impact = impact.at_event.max()
mean_impact = impact.at_event.mean()

print(f"\n📈 STATISTIQUES:")
print("-" * 40)
print(f"Impact maximum: {max_impact/1e9:.2f} Mds FCFA")
print(f"Impact moyen: {mean_impact/1e9:.2f} Mds FCFA")
print(f"Ratio AAI/Impact moyen: {aai_total/mean_impact:.2f}")

# Pourcentage de la valeur exposée
total_exposure = exposures.gdf.value.sum()
damage_ratio = (aai_total / total_exposure) * 100
print(f"\n💸 Dommages annuels attendus: {damage_ratio:.2f}% de la valeur exposée")

## 🗺️ Étape 6 : Visualisations

Créons des cartes pour visualiser nos résultats.

In [ ]:
# Carte 1: Exposition économique par catégorie
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Carte exposition
colors = {'Résidentiel': 'blue', 'Commercial': 'green', 'Industriel': 'red', 'Infrastructure': 'orange'}

for category in categories:
    mask = exposures.gdf.category == category
    if mask.any():
        ax1.scatter(exposures.gdf.longitude[mask], exposures.gdf.latitude[mask], 
                   s=exposures.gdf.value[mask]/1e6*2,  # Taille proportionnelle à la valeur
                   c=colors[category], alpha=0.6, label=category)

ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title('Exposition Économique par Secteur\n(Taille = Valeur)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xlim(lon_min, lon_max)
ax1.set_ylim(lat_min, lat_max)

# Carte intensité aléa (événement le plus intense)
max_event_idx = np.argmax(impact.at_event)
intensity_data = hazard.intensity[max_event_idx, :].toarray().flatten()

scatter = ax2.scatter(centroids_lon, centroids_lat, 
                     c=intensity_data, s=100, cmap='Blues', alpha=0.8)
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude') 
ax2.set_title(f'Intensité Aléa: {hazard.event_name[max_event_idx]}\n(mm/24h)')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(lon_min, lon_max)
ax2.set_ylim(lat_min, lat_max)

# Colorbar
cbar = plt.colorbar(scatter, ax=ax2)
cbar.set_label('Précipitations (mm/24h)')

plt.tight_layout()
plt.show()

In [ ]:
# Graphiques de synthèse
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Impacts par événement
bars = ax1.bar(range(len(hazard.event_name)), impact.at_event/1e9, 
               color=['skyblue', 'lightgreen', 'salmon'])
ax1.set_xlabel('Événements')
ax1.set_ylabel('Impact (Milliards FCFA)')
ax1.set_title('Impact Économique par Événement')
ax1.set_xticks(range(len(hazard.event_name)))
ax1.set_xticklabels([name.replace('_', '\n') for name in hazard.event_name], rotation=45)
ax1.grid(True, alpha=0.3)

# Ajouter valeurs sur barres
for bar, value in zip(bars, impact.at_event/1e9):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{value:.1f}', ha='center', va='bottom')

# 2. Distribution exposition par secteur
sector_values = {}
for cat in categories:
    sector_values[cat] = exposures.gdf[exposures.gdf.category == cat].value.sum() / 1e9

wedges, texts, autotexts = ax2.pie(sector_values.values(), labels=sector_values.keys(), 
                                   autopct='%1.1f%%', startangle=90, 
                                   colors=[colors[cat] for cat in sector_values.keys()])
ax2.set_title('Répartition Exposition par Secteur\n(Milliards FCFA)')

# 3. Courbe de fréquence-impact (exceedance)
sorted_impacts = np.sort(impact.at_event)[::-1]  # Tri décroissant
exceedance_freq = np.arange(1, len(sorted_impacts) + 1) / len(sorted_impacts)
return_periods = 1 / exceedance_freq

ax3.loglog(return_periods, sorted_impacts/1e9, 'bo-', markersize=8, linewidth=2)
ax3.set_xlabel('Période de Retour (années)')
ax3.set_ylabel('Impact (Milliards FCFA)')
ax3.set_title('Courbe de Fréquence-Impact')
ax3.grid(True, which="both", alpha=0.3)

# 4. Comparaison intensité vs impact
max_intensities = hazard.intensity.max(axis=1).toarray().flatten()
ax4.scatter(max_intensities, impact.at_event/1e9, s=100, alpha=0.7, c='purple')
ax4.set_xlabel('Intensité Max (mm/24h)')
ax4.set_ylabel('Impact Total (Milliards FCFA)')
ax4.set_title('Relation Intensité-Impact')
ax4.grid(True, alpha=0.3)

# Ajouter noms événements
for i, name in enumerate(hazard.event_name):
    ax4.annotate(name.replace('_', ' '), 
                (max_intensities[i], impact.at_event[i]/1e9),
                xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.tight_layout()
plt.show()

## 📋 Étape 7 : Analyse et Interprétation

Analysons nos résultats du point de vue de la DGE.

In [ ]:
# Analyse détaillée pour la DGE
print("📊 ANALYSE ÉCONOMIQUE POUR LA DGE")
print("=" * 50)

# 1. Contexte macroéconomique
pib_ci_2023 = 70000e9  # PIB Côte d'Ivoire ~70 000 Mds FCFA (estimation)
pib_abidjan_est = pib_ci_2023 * 0.4  # Abidjan ~40% du PIB national

print(f"\n🏛️ CONTEXTE MACROÉCONOMIQUE:")
print(f"PIB Côte d'Ivoire (2023): {pib_ci_2023/1e12:.0f} billions FCFA")
print(f"PIB Abidjan (estimé): {pib_abidjan_est/1e12:.1f} billions FCFA")
print(f"Exposition analysée: {total_exposure/1e12:.3f} billions FCFA")
print(f"Part échantillon/PIB Abidjan: {(total_exposure/pib_abidjan_est)*100:.2f}%")

# 2. Impacts relatifs
print(f"\n💸 IMPACTS RELATIFS:")
aai_ratio_pib = (aai_total / pib_abidjan_est) * 100
print(f"AAI/PIB Abidjan: {aai_ratio_pib:.3f}%")
print(f"AAI par habitant Abidjan: {aai_total/total_pop:.0f} FCFA/an")

# 3. Scénarios de politique publique
print(f"\n🏗️ IMPLICATIONS POLITIQUES PUBLIQUES:")
print("-" * 40)

# Scénario 1: Investissement drainage
reduction_drainage = 0.3  # 30% réduction impacts
cout_drainage = aai_total * 15  # Coût = 15x bénéfice annuel
benefice_drainage = aai_total * reduction_drainage
roi_drainage = benefice_drainage / cout_drainage

print(f"📋 SCÉNARIO 1 - Amélioration drainage urbain:")
print(f"   Coût investissement: {cout_drainage/1e9:.1f} Mds FCFA")
print(f"   Bénéfice annuel: {benefice_drainage/1e9:.2f} Mds FCFA")
print(f"   ROI annuel: {roi_drainage*100:.1f}%")
print(f"   Période retour: {1/roi_drainage:.1f} ans")

# Scénario 2: Système alerte précoce
reduction_alerte = 0.15  # 15% réduction
cout_alerte = aai_total * 3  # Moins cher
benefice_alerte = aai_total * reduction_alerte
roi_alerte = benefice_alerte / cout_alerte

print(f"\n📋 SCÉNARIO 2 - Système alerte précoce:")
print(f"   Coût investissement: {cout_alerte/1e9:.1f} Mds FCFA")
print(f"   Bénéfice annuel: {benefice_alerte/1e9:.2f} Mds FCFA")
print(f"   ROI annuel: {roi_alerte*100:.1f}%")
print(f"   Période retour: {1/roi_alerte:.1f} ans")

# 4. Recommandations DGE
print(f"\n💡 RECOMMANDATIONS POUR LA DGE:")
print("-" * 40)
print("1. Prioriser système alerte (ROI élevé, coût faible)")
print("2. Planifier drainage urbain (impact fort, ROI acceptable)")
print("3. Intégrer coûts climatiques dans évaluation projets")
print("4. Développer assurance catastrophe naturelle")
print("5. Renforcer normes construction zones à risque")

## 📚 Étape 8 : Exercices Pratiques

### 🎯 Exercice 1: Modification de la fonction de dommage
Modifiez les paramètres de la fonction de dommage et observez l'impact sur les résultats.

In [ ]:
# EXERCICE 1: À vous de jouer!
# Créez une fonction de dommage plus conservative (dommages plus faibles)
# Indice: Réduisez les valeurs de damages_cal de 20%

# Votre code ici:
# ---------------



# ---------------
# Solution cachée (décommenter pour voir):
# damages_cal_conservative = damages_cal * 0.8
# impact_func_conservative = ImpactFunc()
# impact_func_conservative.id = 2
# impact_func_conservative.name = 'Conservative'
# impact_func_conservative.haz_type = 'FL'
# impact_func_conservative.intensity_unit = 'mm/day'
# impact_func_conservative.intensity = intensities_cal
# impact_func_conservative.mdd = damages_cal_conservative / 100
# impact_func_conservative.paa = np.minimum(damages_cal_conservative / 10, 1.0)

### 🎯 Exercice 2: Ajout d'un nouvel événement
Ajoutez un 4ème événement extrême et recalculez les impacts.

In [ ]:
# EXERCICE 2: Ajouter événement "Pluies_Mai_2024" avec intensité 250 mm/24h
# Indice: Modifiez les arrays event_names, event_dates, base_intensities

# Votre code ici:
# ---------------



# ---------------

### 🎯 Exercice 3: Analyse de sensibilité
Étudiez comment varie l'AAI en fonction du nombre de points d'exposition.

In [ ]:
# EXERCICE 3: Analyser sensibilité AAI vs nombre points exposition
# Testez avec n_exposure_points = [50, 100, 200, 500]

# Votre code ici:
# ---------------



# ---------------

## 🎯 Résumé et Points Clés

### ✅ Ce que nous avons appris:

1. **Concepts fondamentaux CLIMADA**:
   - Aléa (Hazard): Précipitations extrêmes spatialisées
   - Exposition (Exposures): Actifs économiques géolocalisés
   - Vulnérabilité (Impact Function): Relation intensité-dommage

2. **Workflow complet**:
   - Import bibliothèques et configuration
   - Création données réalistes pour Côte d'Ivoire
   - Calcul impacts économiques
   - Visualisation et analyse résultats

3. **Applications pratiques DGE**:
   - Évaluation coût-bénéfice investissements
   - Priorisation mesures adaptation
   - Quantification risques économiques

### 🔄 Prochaines étapes:
- TP2: Création d'aléas plus complexes
- TP3: Modélisation exposition détaillée
- TP4: Calibration fonctions dommage locales

### 📞 Support:
En cas de questions: **formation.climada.dge@gouv.ci**

---
**📅 Formation DGE - CLIMADA Côte d'Ivoire 2025**